## Northwind database

Northwind is a company that sells specialty foods. I was given a mandate to  "__do something with this database__" as my first project as a new hire. Unfortunatly at the end of last week, the whole computer science division got sick. It was someone's birthday and they all ate bad sheet cake. I wasn't aware those could *actually* go bad.

I was able to find a entity-relation diagram for the database but not much else. So to break down this task:
1. First I am going to have to explore the database myself to see the basic metrics of the company.
2. Then I will use this basic data to formulate some hypotheses concerning some underlying trends.
3. Finally I will attempt to test out these ideas to prove myself right or wrong.

In [7]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session, sessionmaker #importing libraries I will use. This list has been added as I progressed throughout the notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [12]:
engine = create_engine('sqlite:///Northwind_small.sqlite', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

inspector = inspect(engine) #checking the ERD against the actual database
inspector.get_table_names()

2019-01-24 12:37:28,221 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-01-24 12:37:28,225 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-01-24 12:37:28,227 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

2019-01-24 12:37:28,231 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


2019-01-24 12:37:28,233 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


['Category',
 'Customer',
 'CustomerCustomerDemo',
 'CustomerDemographic',
 'Employee',
 'EmployeeTerritory',
 'Order',
 'OrderDetail',
 'Product',
 'Region',
 'Shipper',
 'Supplier',
 'Territory']

So found my first discrepancy, most the table names are stated as singular while the ERD have them as plural.  
  
I will submit a ticket to the deparment to fix this when they get back.

In [13]:
inspector.get_columns('Product')

2019-01-24 12:50:35,163 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Product")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("Product")


2019-01-24 12:50:35,164 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


[{'name': 'Id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'ProductName',
  'type': VARCHAR(length=8000),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'SupplierId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'CategoryId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'QuantityPerUnit',
  'type': VARCHAR(length=8000),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitPrice',
  'type': DECIMAL(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitsInStock',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'UnitsOnOrder',
  'type': INTEGER(),
  'n

A bit messy, I recall a function that I had used previously that I could use to help in cleaning this up.

In [33]:
def get_column_info(col_name): #quick function based on https://github.com/learn-co-curriculum/dsc-2-13-11-queries-with-sqlalchemy-lab
    col_list = inspector.get_columns(col_name)
    print(f'Table Name: {col_name} \n')
    
    for col in col_list:
        if col['primary_key'] == 1:
            print(f"{col['name']}  ||PRIMARY KEY||  dtype: {col['type']}")
        else:
            print(f"{col['name']}     dtype: {col['type']}")
                  

In [34]:
get_column_info('Product')

Table Name: Product 

Id  ||PRIMARY KEY||  dtype: INTEGER
ProductName     dtype: VARCHAR(8000)
SupplierId     dtype: INTEGER
CategoryId     dtype: INTEGER
QuantityPerUnit     dtype: VARCHAR(8000)
UnitPrice     dtype: DECIMAL
UnitsInStock     dtype: INTEGER
UnitsOnOrder     dtype: INTEGER
ReorderLevel     dtype: INTEGER
Discontinued     dtype: INTEGER


In [36]:
get_column_info('Employee')

2019-01-24 13:14:30,116 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Employee")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("Employee")


2019-01-24 13:14:30,117 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


Table Name: Employee 

Id  ||PRIMARY KEY||  dtype: INTEGER
LastName     dtype: VARCHAR(8000)
FirstName     dtype: VARCHAR(8000)
Title     dtype: VARCHAR(8000)
TitleOfCourtesy     dtype: VARCHAR(8000)
BirthDate     dtype: VARCHAR(8000)
HireDate     dtype: VARCHAR(8000)
Address     dtype: VARCHAR(8000)
City     dtype: VARCHAR(8000)
Region     dtype: VARCHAR(8000)
PostalCode     dtype: VARCHAR(8000)
Country     dtype: VARCHAR(8000)
HomePhone     dtype: VARCHAR(8000)
Extension     dtype: VARCHAR(8000)
Photo     dtype: BLOB
Notes     dtype: VARCHAR(8000)
ReportsTo     dtype: INTEGER
PhotoPath     dtype: VARCHAR(8000)


In [37]:
get_column_info('Supplier')

2019-01-24 13:16:01,343 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Supplier")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("Supplier")


2019-01-24 13:16:01,345 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


Table Name: Supplier 

Id  ||PRIMARY KEY||  dtype: INTEGER
CompanyName     dtype: VARCHAR(8000)
ContactName     dtype: VARCHAR(8000)
ContactTitle     dtype: VARCHAR(8000)
Address     dtype: VARCHAR(8000)
City     dtype: VARCHAR(8000)
Region     dtype: VARCHAR(8000)
PostalCode     dtype: VARCHAR(8000)
Country     dtype: VARCHAR(8000)
Phone     dtype: VARCHAR(8000)
Fax     dtype: VARCHAR(8000)
HomePage     dtype: VARCHAR(8000)


  
So a quick check of the different tables align with the ERD. That is good. But I should be checking out the individual tables before calling them nevertheless, it is good pratice.
  
Now lets start checking out a few things:
- How much of what are we selling?
- Who are our main suppliers/shippers?
- What does our customer base look like?
- What is the geograpical spread of our workforce?

Once we know these things, we will have a broad overview of the business. From there we will investigate any abnormalities or go splunking for underlying trends.
___
---
Now lets make a connect to the engine and make sure it works.

In [38]:
con = engine.connect() #connecting the engine to be able to make queries

In [39]:
q = '''SELECT * FROM Product''' #simple query
df_product = pd.read_sql_query(q, engine) #puts the information from the query into a dataframe
df_product.head()

2019-01-24 13:37:37,666 INFO sqlalchemy.engine.base.Engine SELECT * FROM Product


INFO:sqlalchemy.engine.base.Engine:SELECT * FROM Product


2019-01-24 13:37:37,667 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


Fantasic, now lets start flushing out answers to those inital questions

## How much of what are we selling?

Lets check the Product table and the Order Detail table

In [50]:
print(get_column_info('Product'))
print(get_column_info('OrderDetail'))
print(get_column_info('Category'))

Table Name: Product 

Id  ||PRIMARY KEY||  dtype: INTEGER
ProductName     dtype: VARCHAR(8000)
SupplierId     dtype: INTEGER
CategoryId     dtype: INTEGER
QuantityPerUnit     dtype: VARCHAR(8000)
UnitPrice     dtype: DECIMAL
UnitsInStock     dtype: INTEGER
UnitsOnOrder     dtype: INTEGER
ReorderLevel     dtype: INTEGER
Discontinued     dtype: INTEGER
None
Table Name: OrderDetail 

Id  ||PRIMARY KEY||  dtype: VARCHAR(8000)
OrderId     dtype: INTEGER
ProductId     dtype: INTEGER
UnitPrice     dtype: DECIMAL
Quantity     dtype: INTEGER
Discount     dtype: FLOAT
None
Table Name: Category 

Id  ||PRIMARY KEY||  dtype: INTEGER
CategoryName     dtype: VARCHAR(8000)
Description     dtype: VARCHAR(8000)
None


Looks like the table is ERD is wrong again. Some of the tables are incorrectly stated i.e. ProductID is just Id

In [57]:
q='''SELECT p.ProductName, COUNT(*) num_ordered \
FROM Product p \
INNER JOIN OrderDetail o ON o.ProductId = p.Id \
GROUP BY p.ProductName ORDER BY num_ordered DESC'''
df = pd.read_sql_query(q, engine)
df.head(10)

2019-01-24 14:40:48,937 INFO sqlalchemy.engine.base.Engine SELECT p.ProductName, COUNT(*) num_ordered FROM Product p INNER JOIN OrderDetail o ON o.ProductId = p.Id GROUP BY p.ProductName ORDER BY num_ordered DESC


INFO:sqlalchemy.engine.base.Engine:SELECT p.ProductName, COUNT(*) num_ordered FROM Product p INNER JOIN OrderDetail o ON o.ProductId = p.Id GROUP BY p.ProductName ORDER BY num_ordered DESC


2019-01-24 14:40:48,938 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,ProductName,num_ordered
0,Raclette Courdavault,54
1,Camembert Pierrot,51
2,Gorgonzola Telino,51
3,Guaraná Fantástica,51
4,Gnocchi di nonna Alice,50
5,Tarte au sucre,48
6,Jack's New England Clam Chowder,47
7,Rhönbräu Klosterbier,46
8,Chang,44
9,Pavlova,43


In [55]:
q='''SELECT p.ProductName, c.CategoryName, COUNT(*) num_ordered \
FROM Product p \
INNER JOIN OrderDetail o ON o.ProductId = p.Id \
INNER JOIN Category c ON c.Id = p.Id \
GROUP BY p.ProductName ORDER BY num_ordered DESC'''
df = pd.read_sql_query(q, engine)
df.head()

2019-01-24 14:39:57,971 INFO sqlalchemy.engine.base.Engine SELECT p.ProductName, c.CategoryName, COUNT(*) num_ordered FROM Product p INNER JOIN OrderDetail o ON o.ProductId = p.Id INNER JOIN Category c ON c.Id = p.Id GROUP BY p.ProductName ORDER BY num_ordered DESC


INFO:sqlalchemy.engine.base.Engine:SELECT p.ProductName, c.CategoryName, COUNT(*) num_ordered FROM Product p INNER JOIN OrderDetail o ON o.ProductId = p.Id INNER JOIN Category c ON c.Id = p.Id GROUP BY p.ProductName ORDER BY num_ordered DESC


2019-01-24 14:39:57,972 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,ProductName,CategoryName,num_ordered
0,Chang,Condiments,44
1,Chai,Beverages,38
2,Uncle Bob's Organic Dried Pears,Produce,29
3,Chef Anton's Cajun Seasoning,Dairy Products,20
4,Northwoods Cranberry Sauce,Seafood,13


Next steps: Compare the JOIN statements. An inner join gave me less stuff. WHY? Check out some videos to start your morning off right.